In [5]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
data.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_test = le.transform(Y_test)

In [9]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
Y_train_tensor = torch.from_numpy(Y_train.astype(np.float32))
Y_test_tensor = torch.from_numpy(Y_test.astype(np.float32))

In [11]:
X_train_tensor.shape, X_test_tensor.shape, Y_train_tensor.shape, Y_test_tensor.shape

(torch.Size([455, 30]),
 torch.Size([114, 30]),
 torch.Size([455]),
 torch.Size([114]))

In [17]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    z = self.linear(x)
    return self.sigmoid(z)

  def loss(self, y_pred, Y):
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    loss = Y * torch.log(y_pred) + (1 - Y) * torch.log(1 - y_pred)
    return -torch.mean(loss)

In [18]:
lr = 0.01
epochs = 1000

## using custom loss function and optimizer

In [19]:
model = MyNN(X_train_tensor.shape[1])

for i in range(epochs):
  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculation
  loss = model.loss(y_pred, Y_train_tensor)
  print(f'Epoch: {i + 1} | Loss: {loss.item()}', end='--->')
  if (i + 1) % 100 == 0:
    print()

  # backward pass
  loss.backward()

  # weight update
  with torch.no_grad():
    model.linear.weight -= lr * model.linear.weight.grad
    model.linear.bias -= lr * model.linear.bias.grad

  # zero the gradients
  model.linear.weight.grad.zero_()
  model.linear.bias.grad.zero_()

model.linear.weight, model.linear.bias

Epoch: 1 | Loss: 0.7296302318572998--->Epoch: 2 | Loss: 0.7290014624595642--->Epoch: 3 | Loss: 0.7283914089202881--->Epoch: 4 | Loss: 0.7277989387512207--->Epoch: 5 | Loss: 0.7272233963012695--->Epoch: 6 | Loss: 0.7266639471054077--->Epoch: 7 | Loss: 0.7261197566986084--->Epoch: 8 | Loss: 0.7255903482437134--->Epoch: 9 | Loss: 0.7250745892524719--->Epoch: 10 | Loss: 0.7245721220970154--->Epoch: 11 | Loss: 0.7240822315216064--->Epoch: 12 | Loss: 0.7236045002937317--->Epoch: 13 | Loss: 0.7231382131576538--->Epoch: 14 | Loss: 0.7226828932762146--->Epoch: 15 | Loss: 0.7222379446029663--->Epoch: 16 | Loss: 0.7218030095100403--->Epoch: 17 | Loss: 0.7213776111602783--->Epoch: 18 | Loss: 0.7209613919258118--->Epoch: 19 | Loss: 0.7205538153648376--->Epoch: 20 | Loss: 0.7201545834541321--->Epoch: 21 | Loss: 0.7197633385658264--->Epoch: 22 | Loss: 0.7193797826766968--->Epoch: 23 | Loss: 0.719003438949585--->Epoch: 24 | Loss: 0.7186342477798462--->Epoch: 25 | Loss: 0.7182717323303223--->Epoch: 26 

(Parameter containing:
 tensor([[ 0.1140, -0.0386, -0.1045,  0.1576, -0.0161, -0.0900,  0.0852,  0.0739,
           0.0320, -0.0567, -0.0635, -0.0026, -0.0344,  0.0437,  0.0054,  0.1374,
          -0.0676,  0.0683, -0.0709, -0.0045, -0.0544,  0.0536, -0.1299,  0.0648,
           0.0231, -0.1236, -0.0851, -0.0961,  0.0832,  0.1481]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.4111], requires_grad=True))

In [20]:
with torch.no_grad():
  y_pred = model(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == Y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.6842105388641357


## using built loss function and optim from nn.optim class

In [21]:
class MyNN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    z = self.linear(x)
    return self.sigmoid(z)

In [24]:
loss_function = nn.BCELoss()

In [49]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [54]:
model = MyNN(X_train_tensor.shape[1])

for i in range(1000):
  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculation
  loss = loss_function(y_pred, Y_train_tensor.reshape(-1, 1))
  print(f'Epoch: {i + 1} | Loss: {loss.item()}', end='--->')
  if (i + 1) % 100 == 0:
    print()

  optimizer.zero_grad()
  # backward pass
  loss.backward()

  '''
    # weight update
    with torch.no_grad():
      model.linear.weight -= lr * model.linear.weight.grad
      model.linear.bias -= lr * model.linear.bias.grad

    # zero the gradients
    model.linear.weight.grad.zero_()
    model.linear.bias.grad.zero_()
  '''
  optimizer.step()

model.linear.weight, model.linear.bias

Epoch: 1 | Loss: 0.5073736310005188--->Epoch: 2 | Loss: 0.5073736310005188--->Epoch: 3 | Loss: 0.5073736310005188--->Epoch: 4 | Loss: 0.5073736310005188--->Epoch: 5 | Loss: 0.5073736310005188--->Epoch: 6 | Loss: 0.5073736310005188--->Epoch: 7 | Loss: 0.5073736310005188--->Epoch: 8 | Loss: 0.5073736310005188--->Epoch: 9 | Loss: 0.5073736310005188--->Epoch: 10 | Loss: 0.5073736310005188--->Epoch: 11 | Loss: 0.5073736310005188--->Epoch: 12 | Loss: 0.5073736310005188--->Epoch: 13 | Loss: 0.5073736310005188--->Epoch: 14 | Loss: 0.5073736310005188--->Epoch: 15 | Loss: 0.5073736310005188--->Epoch: 16 | Loss: 0.5073736310005188--->Epoch: 17 | Loss: 0.5073736310005188--->Epoch: 18 | Loss: 0.5073736310005188--->Epoch: 19 | Loss: 0.5073736310005188--->Epoch: 20 | Loss: 0.5073736310005188--->Epoch: 21 | Loss: 0.5073736310005188--->Epoch: 22 | Loss: 0.5073736310005188--->Epoch: 23 | Loss: 0.5073736310005188--->Epoch: 24 | Loss: 0.5073736310005188--->Epoch: 25 | Loss: 0.5073736310005188--->Epoch: 26

(Parameter containing:
 tensor([[ 0.1802,  0.1276,  0.1666, -0.0707, -0.1190,  0.1079, -0.1350,  0.0405,
           0.0086,  0.0983,  0.1724,  0.0623,  0.0853,  0.1600, -0.0561, -0.0789,
           0.1809,  0.0014, -0.0626, -0.1390,  0.0767,  0.1168, -0.0116, -0.1420,
          -0.1257, -0.0787, -0.0462,  0.0834,  0.1705, -0.0364]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.1245], requires_grad=True))

In [55]:
with torch.no_grad():
  y_pred = model(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == Y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5484764575958252
